In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from tqdm import tqdm
from nn_utils import *
import optuna

#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'

In [2]:
def optuna_optimization(trial):

    beta_f = trial.suggest_float('beta_f', 0.5, 0.99)
    beta_r = trial.suggest_float('beta1', 0.5, 0.99)
    lr_r = trial.suggest_float('lr_r', 0, 1.0)
    lr_f = trial.suggest_float('lr_f', 0, 1.0)
    stop_frac = trial.suggest_int('stop_frac', 2, 5)
    avg_frac = trial.suggest_int('avg_frac', 2, 5)

    regrets, regrets_mix = run_algorithm(d=d, r=r, m=m, times=times, d_sigma=d_sigma,
                                         beta_f=beta_f, beta_r=beta_r, T_r=T, lr_r=lr_r, lr_f=lr_f,
                                         avg_frac_r=avg_frac, stop_frac_r=stop_frac)

    return sum(abs(np.log(regrets.min(axis=1) / regrets_mix)))


def plot_regrets(res, label, title=None, path=None):
    if title is None:
        plt.title(f"Regret vs Iteration")
    else:
        plt.title(title)
    plt.grid()
    plt.xlabel(f'#Iteration')
    plt.ylabel(f'Regret')
    plt.plot(res.T, label=label)
    plt.legend()
    if path is None:
        plt.show()
    else:
        plt.savefig(f"{path}_regrets.png")
        plt.clf()

    plt.plot(range(res.shape[1]), res.mean(axis=0), label=label)
    plt.fill_between(range(res.shape[1]),
                     res.mean(axis=0)+res.std(axis=0),
                     res.mean(axis=0)-res.std(axis=0),
                     color='r', alpha=0.5)
    plt.legend()
    plt.grid()
    plt.xlabel(f'#Iteration')
    plt.ylabel(f'Regret')
    if path is None:
        plt.show()
    else:
        plt.savefig(f"{path}_avg.png")
        plt.clf()

In [3]:
d=20
r=3
T=100
m=20
times=10
d_sigma=1

"""
sigma = torch.diag(torch.tensor([9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0]))

R, f, curr_regrets = algorithm(sigma=sigma, m=m, r=r,
                                       beta_f=0.95, beta_r=0.95, lr_r=1e-1,
                                       T_r=T, avg_frac_r=4, stop_frac_r=4)
"""

'\nsigma = torch.diag(torch.tensor([9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0]))\n\nR, f, curr_regrets = algorithm(sigma=sigma, m=m, r=r,\n                                       beta_f=0.95, beta_r=0.95, lr_r=1e-1,\n                                       T_r=T, avg_frac_r=4, stop_frac_r=4)\n'

In [4]:
name = f"optuna-nn_d={d}_r={r}_m={m}_Tr={T}_var={d_sigma}"
study = optuna.create_study(study_name=f'{name}',
                            storage=f'sqlite:///./../optuna/{name}.db ',
                            direction='minimize', load_if_exists=True)
study.optimize(optuna_optimization, n_trials=100)


[I 2023-05-15 13:05:13,849] Using an existing study with name 'optuna-nn_d=20_r=3_m=20_Tr=100_var=1' instead of creating a new one.


l_star=6 ; regret_mix=1.8152978177538177 ; 

100%|██████████| 100/100 [00:05<00:00, 17.78it/s]
[W 2023-05-15 13:11:26,202] Trial 5 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "c:\users\neria\pycharmprojects\aofbot\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-2-78eb67eb14b3>", line 12, in optuna_optimization
    avg_frac_r=avg_frac, stop_frac_r=stop_frac)
  File "C:\Users\neria\Desktop\masters\thesis material\paper-optimization-algorithm\mse\nn_utils.py", line 261, in run_algorithm
    T_r=T_r, avg_frac_r=avg_frac_r, stop_frac_r=stop_frac_r)
  File "C:\Users\neria\Desktop\masters\thesis material\paper-optimization-algorithm\mse\nn_utils.py", line 224, in algorithm
    beta_r=beta_r, beta_f=beta_f, lr_r=lr_r, lr_f=lr_f, avg_frac_r=avg_frac_r, stop_frac_r=stop_frac_r)
  File "C:\Users\neria\Desktop\masters\thesis material\paper-optimization-algorithm\mse\nn_utils.py", line 207, in init_R0_f0
    

KeyboardInterrupt: 